# Pokémon TCG Synergy Engine
---

In [1]:
# import libraries for data wrangling
import ast
import re
import pandas as pd
import numpy as np
import src

## Pokémon TCG Data Wrangling
---

In [2]:
data = src.get_card_data()

In [3]:
df_pokemon_cards = src.prep_card_data(data)

In [4]:
df_pokemon_cards

,abilities,attacks,retreat_cost,evolves_from,hp,id,legalities,regulation_mark,name,number,...,standard_legality,ability_name,ability_text,attack_name,attack_text,attack_damage,attack_cost,attack_energy_cost,stage,setup_time
0,"[{'name': 'Adaptive Evolution', 'text': 'This ...","[{'name': 'Tackle', 'cost': ['Grass', 'Colorle...",1.0,NaN,30.0,sv1-8,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Scatterbug,8,...,Legal,Adaptive Evolution,This Pokémon can evolve during your first turn...,Tackle,,20,"[Grass, Colorless]",2,Basic,0
1,NaN,"[{'name': 'Guard Press', 'cost': ['Colorless',...",2.0,NaN,60.0,sv1-1,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Pineco,1,...,Legal,NaN,NaN,Guard Press,"During your opponent's next turn, this Pokémon...",10,"[Colorless, Colorless]",2,Basic,0
2,"[{'name': 'Counterattack Quills', 'text': 'If ...","[{'name': 'Spike Shot', 'cost': ['Colorless', ...",2.0,Cacnea,130.0,sv1-6,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Cacturne,6,...,Legal,Counterattack Quills,If this Pokémon is in the Active Spot and is d...,Spike Shot,,110,"[Colorless, Colorless, Colorless]",3,Stage 1,1
3,NaN,"[{'name': 'Rising Lunge', 'cost': ['Colorless'...",2.0,Skiddo,130.0,sv1-12,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Gogoat,12,...,Legal,NaN,NaN,Rising Lunge,"Flip a coin. If heads, this attack does 30 mor...",30+,"[Colorless, Colorless]",2,Stage 1,1
4,NaN,"[{'name': 'Rising Lunge', 'cost': ['Colorless'...",2.0,Skiddo,130.0,sv1-12,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Gogoat,12,...,Legal,NaN,NaN,Solar Beam,,110,"[Grass, Colorless, Colorless]",3,Stage 1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,NaN,"[{'name': 'Add On', 'cost': ['Colorless'], 'co...",NaN,Taillow,100.0,sv10-157,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",I,Swellow,157,...,Legal,NaN,NaN,Add On,Draw 3 cards.,,[Colorless],1,Stage 1,1
2605,NaN,"[{'name': 'Add On', 'cost': ['Colorless'], 'co...",NaN,Taillow,100.0,sv10-157,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",I,Swellow,157,...,Legal,NaN,NaN,Speed Wing,,70,"[Colorless, Colorless]",2,Stage 1,1
2606,NaN,"[{'name': 'Gnaw Through', 'cost': ['Colorless'...",1.0,NaN,60.0,sv10-158,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",I,Arven's Skwovet,158,...,Legal,NaN,NaN,Gnaw Through,"Before doing damage, discard all Pokémon Tools...",10,[Colorless],1,Basic,0
2607,"[{'name': 'Greedy Order', 'text': 'When you pl...","[{'name': 'Rolling Tackle', 'cost': ['Colorles...",2.0,Arven's Skwovet,120.0,sv10-159,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",I,Arven's Greedent,159,...,Legal,Greedy Order,When you play this Pokémon from your hand to e...,Rolling Tackle,,50,"[Colorless, Colorless]",2,Stage 1,1


In [5]:
# Create features to indicate if a card is ex and or tera typed.
df_pokemon_cards['is_ex'] = df_pokemon_cards['subtypes'].apply(lambda x: 1 if 'ex' in x else 0)
df_pokemon_cards['is_tera'] = df_pokemon_cards['subtypes'].apply(lambda x: 1 if 'Tera' in x else 0)

In [6]:
df_pokemon_cards['primary_type'] = df_pokemon_cards['types'].apply(
    lambda x: ast.literal_eval(x)[0] if pd.notnull(x) else x
)

In [7]:
df_pokemon_cards['release_date'] = df_pokemon_cards['set'].apply(lambda x: x.get('releaseDate') if isinstance(x, dict) else None)
df_pokemon_cards['release_date'] = pd.to_datetime(df_pokemon_cards['release_date'], errors='coerce')

# Step 2: Extract the year
df_pokemon_cards['release_year'] = df_pokemon_cards['release_date'].dt.year

In [9]:
def extract_prize_value(rule):
    # Handle missing or empty rule values
    if rule is None or rule == '' or rule == []:
        return 1
    
    # If the rule is a stringified list, convert it to an actual list
    if isinstance(rule, str):
        try:
            rule = ast.literal_eval(rule)
        except Exception:
            return 1

    # At this point, rule should be a list
    if isinstance(rule, list):
        for r in rule:
            match = re.search(r'takes (\d+) Prize', r)
            if match:
                return int(match.group(1))

    return 1 

# Apply it to your dataframe
df_pokemon_cards['prize_card_value'] = df_pokemon_cards['rules'].apply(extract_prize_value)


In [10]:
df_pokemon_cards['prize_card_value'].value_counts()

prize_card_value
1    2274
2     335
Name: count, dtype: int64

In [11]:
# Create the feature flag for bench damage immunity
df_pokemon_cards['is_immune_to_bench_damage'] = df_pokemon_cards['rules'].apply(
    lambda x: int(any('As long as this Pokémon is on your Bench, prevent all damage done' in rule for rule in x)) if isinstance(x, list) else 0
)

In [12]:
df_pokemon_cards['attack_damage_amount'] = df_pokemon_cards['attack_damage'].str.extract('([0-9]*)')
df_pokemon_cards['attack_damage_modifier'] = df_pokemon_cards['attack_damage'].str.replace('([0-9])', '')

In [13]:
df_pokemon_cards['cards_needed_for_attack'] = df_pokemon_cards['setup_time'] + df_pokemon_cards['attack_energy_cost']

In [14]:
cols_to_keep = ['id',
 'supertype',
 'name',
 'stage',
 'is_ex',
 'is_tera',
 'primary_type',
 'evolves_from',
 'hp',
 'ability_name',
 'ability_text',
 'attack_name',
 'attack_text',
 'attack_damage_amount',
 'attack_damage_modifier',
 'attack_cost',
 'cards_needed_for_attack',
 'attack_energy_cost',
 'retreat_cost',
 'regulation_mark',
 'prize_card_value',
 'setup_time',
 'is_immune_to_bench_damage',
 'release_date',
 'release_year'
]

In [15]:
df_pokemon_cards = df_pokemon_cards[cols_to_keep]

In [16]:
df_pokemon_cards['attack_damage_amount'] = pd.to_numeric(df_pokemon_cards['attack_damage_amount'], errors='coerce')
df_pokemon_cards['is_coin_flip'] = df_pokemon_cards['attack_text'].str.contains('coin')

/var/folders/n1/4wvxl50j3n3g13qp1jgnh94w0000gn/T/ipykernel_6957/4087825855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pokemon_cards['attack_damage_amount'] = pd.to_numeric(df_pokemon_cards['attack_damage_amount'], errors='coerce')
/var/folders/n1/4wvxl50j3n3g13qp1jgnh94w0000gn/T/ipykernel_6957/4087825855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pokemon_cards['is_coin_flip'] = df_pokemon_cards['attack_text'].str.contains('coin')


In [18]:
df_pokemon_cards['damage_per_energy'] = np.where(
    df_pokemon_cards['attack_energy_cost'] == 0,
    np.nan,  # or 0 if you prefer
    round(df_pokemon_cards['attack_damage_amount'] / df_pokemon_cards['attack_energy_cost'], 2)
)

df_pokemon_cards['damage_per_energy'] = pd.to_numeric(df_pokemon_cards['damage_per_energy'], errors='coerce')

/var/folders/n1/4wvxl50j3n3g13qp1jgnh94w0000gn/T/ipykernel_6957/112304812.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pokemon_cards['damage_per_energy'] = np.where(
/var/folders/n1/4wvxl50j3n3g13qp1jgnh94w0000gn/T/ipykernel_6957/112304812.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pokemon_cards['damage_per_energy'] = pd.to_numeric(df_pokemon_cards['damage_per_energy'], errors='coerce')


In [19]:
df_pokemon_cards.to_csv('pokemon_cleaned.csv', index=False)

In [20]:
df_pokemon_cards

,id,supertype,name,stage,is_ex,is_tera,primary_type,evolves_from,hp,ability_name,...,attack_energy_cost,retreat_cost,regulation_mark,prize_card_value,setup_time,is_immune_to_bench_damage,release_date,release_year,is_coin_flip,damage_per_energy
0,sv1-8,Pokémon,Scatterbug,Basic,0,0,Grass,NaN,30.0,Adaptive Evolution,...,2,1.0,G,1,0,0,2023-03-31,2023,False,10.00
1,sv1-1,Pokémon,Pineco,Basic,0,0,Grass,NaN,60.0,NaN,...,2,2.0,G,1,0,0,2023-03-31,2023,False,5.00
2,sv1-6,Pokémon,Cacturne,Stage 1,0,0,Grass,Cacnea,130.0,Counterattack Quills,...,3,2.0,G,1,1,0,2023-03-31,2023,False,36.67
3,sv1-12,Pokémon,Gogoat,Stage 1,0,0,Grass,Skiddo,130.0,NaN,...,2,2.0,G,1,1,0,2023-03-31,2023,True,15.00
4,sv1-12,Pokémon,Gogoat,Stage 1,0,0,Grass,Skiddo,130.0,NaN,...,3,2.0,G,1,1,0,2023-03-31,2023,False,36.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,sv10-157,Pokémon,Swellow,Stage 1,0,0,Colorless,Taillow,100.0,NaN,...,1,NaN,I,1,1,0,2025-05-30,2025,False,NaN
2605,sv10-157,Pokémon,Swellow,Stage 1,0,0,Colorless,Taillow,100.0,NaN,...,2,NaN,I,1,1,0,2025-05-30,2025,False,35.00
2606,sv10-158,Pokémon,Arven's Skwovet,Basic,0,0,Colorless,NaN,60.0,NaN,...,1,1.0,I,1,0,0,2025-05-30,2025,False,10.00
2607,sv10-159,Pokémon,Arven's Greedent,Stage 1,0,0,Colorless,Arven's Skwovet,120.0,Greedy Order,...,2,2.0,I,1,1,0,2025-05-30,2025,False,25.00
